<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# Introduction to Image Segmentation with TensorFlow

有很多重要的图像分析的深度学习应用，不仅需要检测一张图像中的单个对象，还需要将图像分割成多个感兴趣的空间区域。例如，在医学影像分析中，经常需要根据不同的组织、血液或异常细胞类型区分像素，以便能够分离出特定的器官。在这个自定进度的动手实验中，我们将利用 [TensorFlow](https://www.tensorflow.org)机器学习框架，借助医学影像数据集来训练及评估图像分割网络模型。

实验由 Jonathan Bentz 创建 (在 Twitter 上关注 [@jnbntz](https://twitter.com/jnbntz))。

---
开始之前，我们先来验证 [WebSockets](http://en.wikipedia.org/wiki/WebSocket) 能否在您的系统中正常工作。要执行此操作，请将焦点切换到下面的单元块代码中（使用鼠标点击它）并按下 Ctrl-Enter，或点击工具栏上的“play”（播放）按钮以执行该单元块。如果一切顺利，您应该在灰色单元下看到返回的一些输出内容。否则，请参阅 [Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) 以解决问题。

In [ ]:
print "The answer should be three: " + str(1+2)

让我们执行下面的单元，显示服务器上GPU的运行信息。

In [ ]:
!nvidia-smi

最后，执行下一个单元，显示本实验中所使用的 TensorFlow 的版本。

In [ ]:
!python -c 'import tensorflow as tf; print(tf.__version__)'

如果您之前未使用过 NVIDIA 基于 IPython Notebook 的自定进度实验，建议您观看这段简短的 [YouTube 视频](http://www.youtube.com/embed/ZMrDaLSFqpY).

另外，如果您是高级用户并偏好使用终端学习本实验，可以单击 [终端]((../../../../terminals/1)，然后以此方式运行所有实验练习。

# 图像分割

在该实验中，您将完成一系列图像分割（也称作“语义分割”）的练习。语义分割指的任务是将各像素归入特定类别。从某种意义上讲，这是一个分类问题，您需要按像素而非整个图像来归类。在本实验中，该任务是依据像素是否属于左心室 (LV) 的一部分来对心脏核磁共振 (MRI ) 图像中的各个像素进行分类。

本实验既非深度学习简介，也非卷积神经网络的严谨数学形式解读。我们会假定您对神经网络至少已有一定程度的了解，包括前向传播和反向传播、激活函数、SGD、卷积、池化、偏差等概念。如果您已经接触过卷积神经网络 (CNN) 并且了解图像识别任务，那会非常有帮助。本实验将使用 Google 的 TensorFlow 机器学习框架，如果您有使用 Python 和 TensorFlow 的经验，也会非常有帮助，但这并非必要条件。本实验中要做的大部分工作并非编码，而是使用 TensorFlow 建立并运行训练和评估任务。


## 输入数据集

您将要使用的数据集是经过专业标注的一系列心脏图像（具体而言是 MRI 短轴 (SAX) 扫描图像）。请参阅 [[1](#1), [2](#2), [3](#3)] ，了解完整的引用信息。

下面是四个具有代表性的数据示例。每行图像是一个数据实例。左侧是 MRI 图像，右侧是经专业分割的区域（常称作“轮廓”）。图像中属于 LV 的部分以白色表示。请注意，LV 的大小因图像而异，但 LV 通常只占整个图像中较小的区域。

***
![Figure 1](images/fig1_final.png)
***
![Figure 2](images/fig2_final.png)
***
![Figure 3](images/fig3_final.png)
***
![Figure 4](images/fig4_final.png)
***

关于如何从原始图像中提取数据以及如何准备这些图像并输入至 TensorFlow，并不是本实验的演示范围。简单来说，数据准备是机器学习工作流程中一个非常重要的方面，但并不属于本实验的范围。
若有人有兴趣了解详细信息，可以参阅来自某一次 [Kaggle竞赛](https://www.kaggle.com/c/second-annual-data-science-bowl/details/deep-learning-tutorial) 的指南及部分代码，了解如何正确地提取图像。我们就是从那拿到这些图像，并将其转换成 TensorFlow 记录 (TFRecords)，然后存储到文件中。[TFRecords](https://www.tensorflow.org/programmers_guide/reading_data) 是 TensorFlow 提供的一种特殊的文件格式，允许您使用内置的 TensorFlow 函数执行数据管理，包括多线程数据读取和较复杂的数据预处理（例如随机化），乃至增加训练数据。

这些图像最初是 256 x 256 灰度[DICOM](https://en.wikipedia.org/wiki/DICOM)格式，这是医学成像中常见的一种图像格式。标签是一个大小为 256 x 256 x 2 的张量。最后一个维数之所以是 2，是因为像素会归入两个类别之一，所以每个像素标签有一个与之相关的大小为 2 的向量。该训练集中有 234 个图像，验证集（并非用于训练而是用于测试模型准确性的数据）中有 26 个图像。

# 利用 TensorFlow 进行深度学习

本实验属于自定进度实验系列，主要介绍当前已公开发布的一些深度学习框架。TensorFlow 是由[Google](https://www.google.com)公司开发的框架，目前 Google 公司内部的很多研究人员及产品部门人员都在使用。

TensorFlow 是一个面向机器智能的开源软件库。计算指令表示为使用张量（即 tensor，其得名于此）运算的数据流图。如果您能以这种方式表达自己的计算指令，就可以在 TensorFlow 框架中运行您的算法。

TensorFlow 的便捷性体现在支持 CPU 和 GPU 运行，可使用工作站、服务器，甚至在移动平台上部署模型。目前，TensorFlow 支持两种编程语言（Python 或 C++），并且还提供针对其他[语言](https://www.tensorflow.org/api_docs/)的各种支持。TensorFlow 的一个经典应用是，在 Python 中执行训练和测试，完成模型后就可以用 C++ 部署。

TensorFlow 专为提升 CPU 和 GPU 的性能表现而设计。单独执行 TensorFlow 时，您会享有很高的灵活性，您可以在必要时将不同任务明确地分配给 CPU 和 GPU。在 GPU 上运行时，TensorFlow 可以使用包括[cuDNN](https://developer.nvidia.com/cudnn)在内的很多 GPU 库，使最新的 GPU 尽可能地发挥出最佳性能。

帮助您初步熟悉 TensorFlow 是本实验的目的之一。在本实验中，我们无法讨论 TensorFlow 的所有功能特性和选项，但希望您在完成本实验后，可以了解如何使用 TensorFlow 解决具体的机器学习问题，并在这方面有不错的心得。

如需有关 TensorFlow 的综合文档，我们推荐您使用[TensorFlow网站](https://www.tensorflow.org), the [白皮书](http://download.tensorflow.org/paper/whitepaper2015.pdf)及[GitHub网站](https://github.com/tensorflow/tensorflow).

# TensorFlow 基础知识

TensorFlow 有多种用法，您可以根据自己的喜好选择。如果要设计训练任务，常见的方法是使用 TensorFlow Python API。要在 TensorFlow 上执行机器学习训练任务，（至少）有两个不同的步骤。

## 数据流图

首先，您要构建一个数据流图，这是您要执行的计算指令的规则和顺序。借助 TensorFlow API，您可以用 TensorFlow 自带的任何运算（例如卷积、激活函数、池化等）逐层构建神经网络。在构建数据流图的这个阶段，并不会对数据做任何实际计算，只是构建您指定的数据流图。

构建数据流图时，必须指定图变量（`Variable`）。指定一个参数为变量，相当于告诉 TensorFlow 这是要“学习”的参数，即这是一个会随着训练的继续而更新的权重。

## 会话

一旦将神经网络定义为数据流图，您就发起了一个会话(`Session`)。通过这个机制，您可以为先前构建的数据流图指定输入数据和训练参数，然后继续计算。 

通常，当您想修改数据流图时，就会重复这两个步骤，即您会更新数据流图，然后发起新会话。

# 工作流程范例

训练及评估模型的工作流程如下所示：

1.	准备输入数据--输入数据可以是 Numpy 阵列，但如果是非常大的数据集，TensorFlow 会提供专门的格式，叫作 TFRecords。
2.	构建数据流图--创建自己的神经网络数据流图，包括专门的节点，例如推理、损失和训练节点。
3.	训练模型--在 TensorFlow 会话中将输入数据导入数据流图中，对您的输入数据执行循环操作。自定义batch_size、训练次数epochs、学习率等参数。
4.	评估模型--对先前不可见的数据执行推理计算（使用训练中的相同数据流图），并依据适当指标评估模型的准确性。


# TensorBoard

TensorFlow 有一个功能非常丰富的工具，叫作 [TensorBoard](https://www.tensorflow.org/get_started/summaries_and_tensorboard)，可以将程序的很多方面可视化。在 TensorBoard 中，您可以看到数据流图的直观表示，可以绘制不同指标的曲线图，例如损失值、准确率和学习率。基本上，在执行 TensorFlow 期间生成的任何数据都可以通过 TensorBoard 以可视化的方式显示出来，只需在程序中多调用几次 API 即可。

例如，请参考下面的代码片段，创建的是带一个隐藏层的神经网络（现在不用担心代码的细节）。

```
with tf.name_scope('Hidden1'):
    W_fc = tf.Variable(tf.truncated_normal( [256*256, 512],
                 stddev=0.1, dtype=tf.float32), name='W_fc')
    flatten1_op = tf.reshape( images_re, [-1, 256*256])
    h_fc1 = tf.matmul( flatten1_op, W_fc )

with tf.name_scope('Final'):
    W_fc2 = tf.Variable(tf.truncated_normal( [512, 256*256*2],
                stddev=0.1, dtype=tf.float32), name='W_fc2' )
    h_fc2 = tf.matmul( h_fc1, W_fc2 )
    h_fc2_re = tf.reshape( h_fc2, [-1, 256, 256, 2] )

return h_fc2_re
```

TensorBoard 会以下图的形式显示该神经网络。如果细看，您会发现边沿输出了张量维数，也就是说，当您在节点之间移动时，您可以跟踪整个数据流图中的张量数据及其大小的变化。

![TensorBoard Example](images/hidden1.png)

# 任务 1 -- 一个隐藏层

![NN](images/NN.png)

我们要思考的第一个任务是创建一个带一个隐藏层的全连接神经网络，并进行训练和评估。输入该神经网络的数据是每个像素的值，即一个大小为 256 x 256（或 65,536）的阵列。隐藏层的大小可以调节，输出数据是一个大小为 256 x 256 x 2 的阵列，即输入的每个像素可以属于两个类别中的任意一个，因此与每个像素相关的输出值就是该像素归入该特定类别的概率。在这个例子中，两个类别为属于 LV 和不属于 LV。我们会通过一个名为[`sparse_softmax_cross_entropy_with_logits`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#sparse_softmax_cross_entropy_with_logits) 的 TensorFlow 函数计算损失，该函数将 Softmax 与交叉熵计算相结合，并在一次函数调用中完成。

## 训练

因为是第一次练习，代码已经写好。要开始神经网络训练任务，请执行下面的单元。

In [ ]:
!python exercises/simple/runTraining.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train

如果一切正常，您可以在屏幕上看到一些消息。其中有些是 TensorFlow 的通知消息，通常可以忽略。您会希望看到以 “OUTPUT”开头的代码行，这是我们在程序中插入的代码行，专门用于输出特定时间点的信息，例如每 100 步时计算出的损失。您会看到的最后一个行是：

`OUTPUT: Done training for 1 epochs, 231 steps`.  

这表示您的训练工作是一次历经所有训练数据的完整训练。

# 评估

在训练好模型后，我们还需要评估其在处理从未见过的数据时的表现。要评估训练好的模型，可以执行下面的单元。

In [ ]:
!python exercises/simple/runEval.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
        --eval_dir /dli/tasks/tensorboard/eval

同样地，您可以忽略 TensorFlow 的大多数输出，只关注以“OUTPUT”开头的代码行。我在运行时获得的输出数据如下所示。您看到的应该与此相似。

```
OUTPUT: 2017-01-26 17:12:28.929741: precision = 0.503
OUTPUT: 26 images evaluated from file /dli/data/img_segmentation/val_images.tfrecords
```

最后的输出行显示模型的准确性，即模型对每个像素属于（或不属于） LV 的预测对比实际情况的准确度。在上面的例子中，0.503 表示 50.3%，也就是说模型预测每个像素所属类别的正确率约为一半。这个结果算不上很好，但考虑到我们只运行了一个非常简单的网络并且只训练了一次，所以也不算糟糕。

## TensorBoard

这时，如果您还没有启动 TensorBoard，就可以在这里启动。[TensorBoard](/tensorboard/)  有很多令人赞叹的可视化功能。顶部菜单栏中有一个名为“Scalars” 的菜单，点击该菜单，可以显示部分已经捕获的信息。您可以点击展开其中的任意信息，查看该数据的图形曲线。

顶部还有一个菜单选项叫“Graphs”。如果选择该选项，可以查看训练及评估的数据流图。数据流图中的每个节点都可以点击展开，您可以从此获取更多有关该节点的详细信息。在该页面的左上角，通过一个小的下拉式选项可以选择是否查看训练或评估的数据流图。

此任务的代码如下：


```
with tf.name_scope('Hidden1'):
    W_fc = tf.Variable(tf.truncated_normal( [256*256, 512],
                 stddev=0.1, dtype=tf.float32), name='W_fc')
    flatten1_op = tf.reshape( images_re, [-1, 256*256])
    h_fc1 = tf.matmul( flatten1_op, W_fc )

with tf.name_scope('Final'):
    W_fc2 = tf.Variable(tf.truncated_normal( [512, 256*256*2],
                stddev=0.1, dtype=tf.float32), name='W_fc2' )
    h_fc2 = tf.matmul( h_fc1, W_fc2 )
    h_fc2_re = tf.reshape( h_fc2, [-1, 256, 256, 2] )

return h_fc2_re

```

可以看出，这是 Python 语法，带有部分 TensorFlow API 调用。
* `tf.name_scope()` 允许您指定程序的具体范围。在组织代码和命名 TensorBoard 中的数据流图中的节点时非常有用。
* `tf.Variable()` 指定要训练的 TensorFlow 图变量，即这是一个权重张量。
* `tf.reshape()` 是一个 TensorFlow 辅助函数，用于调整张量的形状，令其适合稍后张量运算的正确形状。
* `tf.matmul()` 正如您所想的那样，是两个 TensorFlow 张量的矩阵相乘。

您可以看到，每项运算都以显式方式调用，即定义可作权重训练的`tf.Variable`张量，以及将权重与输入结合以构建神经网络的运算，如`tf.matmul()`。虽然这为您准确定义所需神经网络的类型提供了最大灵活性，但在您开始添加激活函数、偏差等类似内容后，即使相对简单的神经网络的创建也可能变得非常复杂。

为了减轻部分类似工作，TensorFlow 提供了更高级别的[Layers API](https://www.tensorflow.org/tutorials/layers)，它包含常用的层类型（如，卷积、池化、全连接等），同时内置激活、偏差、正则化等层。使用层 API 时，可以通过非常简洁且简单的方式在 TensorFlow 中构建神经网络。对于我们的后续练习，我们将使用层 API 来构建神经网络。

### 未详细讨论的主题

我们略过了很多主题，考虑到完整性，我们会简单提及，但不会详细讨论。
* 我们假定所有数据已经设置妥当。如前所示，我们使用的是已经设置好的 TFRecords 文件数据格式。
* 我们使用 TensorFlow 机制，通过多个线程读取这些文件中的数据。这允许我们使用内置的 TensorFlow 函数来完成数据随机化, 并自动处理`batch_size`和`num_epochs`等。
* 我们只简要讨论了如何通过数据流图实际构建模型。这是大量 Python 语法，如果愿意，您可以在代码中查看。
* 最后，我们插入特殊的 API 调用，以导出数据至 TensorBoard，以便可以轻松地绘制和查看。同样地，这是 Python 的样板代码，如果愿意，您也可以查看。


# 任务 2 -- 卷积神经网络 (CNN)

我们的第二个任务是将模型转换为较高级的网络，包含比上一个网络更多的层和类型。上一个范例侧重的是每个单独的像素，无法考虑到感兴趣区域很可能比单个像素更大的情况。我们还想捕获较小的感兴趣区域，为此，我们会用到卷积层，它也能捕获较大的感受野。

我们还会增加池化层，它可以对数据执行下采样，同时尽量保留大部分信息。这样可以避免部分计算复杂性。

目前为止，我们介绍了常常与图像识别神经网络相关的层，其中的输出节点数等于类别数。回顾一下，我们所做的不单是图像分类，而是对图像中的每一个像素进行归类，所以我们的输出大小应为类别数 (2) 乘像素数 (256 x 256)。此外，输出节点的空间位置也很重要，因为每个像素都有属于（或不属于） LV 一部分的相关概率。

CNN 是执行图像识别或分类任务的公认佳选。我们在本实验中的任务是分割，从某种意义上说，这与分类有关。我们可以对图像中的每个像素分类，而不是对整个图像分类。问题在于，我们能否使用在图像识别方面已知的表现出色的同类型 CNN 来执行图像分割任务？事实表明，我们只要对 CNN 模型做些修改即可。
我们可以使用标准的图像识别神经网络，并将全连接层（通常是最后几个层）代之以反卷积层（或许更准确的叫法是
 [transpose convolution](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#conv2d_transpose)层），通过这种方式完成这项任务。

反卷积是一种上采样方法，可以将较小的图像数据还原至原始大小，以便进行最后的图像分类。关于这个主题，我们推荐一些优秀的资源：[[4](#4), [5](#5), [6](#6)]。在修改 CNN网络 令其适合图像分割任务时，得到的神经网络通常称为全卷积神经网络或 FCN。

它可以用来直观地呈现输入数据（在我们的例子中，即大小为 256 x 256 x 1 的一个张量）如何“流”经数据流图，也就是说，数据如何通过卷积和池化等不同运算完成转变。下图表示数据在下一个任务中将会经历的转变。

![FCN](images/FCN.png)

上述数字代表的网络与参考资料 [[7](#7)]中的网络相似。它由卷积层、池化层和最后的反卷积层组成，输入图像正在经历如图中所示的转变。 

这项任务需要您先完成该神经网络，然后进行训练。要完成这项任务，可以编辑文件[`exercises/cnn/neuralnetwork.py`](../../../../edit/tasks/task1/task/exercises/cnn/neuralnetwork.py) ，并以代码替换`FIXME`的所有内容。这里有些代码说明可以提供帮助，您也可以使用下面的网络结构。当您检查并完成代码时，您会更理解这些层名。

1.	Convolution1，5 x 5 内核，步长 2
2.	Maxpooling1，2 x 2 窗口，步长 2
3.	Convolution2，5 x 5 内核，步长 2
4.	Maxpooling2，2 x 2 窗口，步长 2
5.	Convolution3，3 x 3 内核，步长 1
6.	Convolution4，3 x 3 内核，步长 1
7.	Score_classes，1x1 内核，步长 1
8.	Upscore（反卷积），31 x 31 内核，步长 16


如果您想检查自己的代码，可以查看该解决方案： [`exercise_solutions/cnn/neuralnetwork.py`](../../../../edit/tasks/task1/task/exercise_solutions/cnn/neuralnetwork.py).

完成代码后，您可以使用下面的框执行训练，并通过在上一个任务中打开的 TensorBoard窗口可视化结果。如果没有立即看到结果，或须等待片刻，以便 TensorBoard 识别有新数据流图有待可视化。您也可能需要刷新浏览器。

In [ ]:
!python exercises/cnn/runTraining.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
        --num_epochs 1

训练结束后，可以执行如下单元，查看模型的准确性。

In [ ]:
!python exercises/cnn/runEval.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
        --eval_dir /dli/tasks/tensorboard/eval

在 `runTraining.py`  命令中，您可以运行上面的两个单元，新增一些命令行参数，以测试不同的训练参数。如果有时间，可以更改 `--num_epochs`参数来实验，看看这对训练的准确性有何影响。

您可以使用的命令行参数完整列表如下所示。

```
optional arguments:
  -h, --help            show this help message and exit
  --learning_rate LEARNING_RATE
                        Initial learning rate.
  --decay_rate DECAY_RATE
                        Learning rate decay.
  --decay_steps DECAY_STEPS
                        Steps at each learning rate.
  --num_epochs NUM_EPOCHS
                        Number of epochs to run trainer.
  --data_dir DATA_DIR   Directory with the training data.
  --checkpoint_dir CHECKPOINT_DIR
                        Directory where to write model checkpoints.
```

注：如果查看源代码，还会看到一个修改batch_size的选项。在本实验中，请将batch_size设定为 1。

您可以得到的最佳准确性是多少？举例而言，训练 1 次，我们会得到 56.7% 的准确性：

```
OUTPUT: 2017-01-27 17:41:52.015709: precision = 0.567
```

如果将训练次数增至 30，我们可以得到更高的准确性：

```
OUTPUT: 2017-01-27 17:47:59.604529: precision = 0.983
```

可以看出，当增加训练次数时，准确性也会有显著提高。实际上，能达到 98.3% 的准确性就已经很好了。这个准确性足够吗？我们完成了吗？

# 准确性

在讨论准确性时，我们需要退后一步全面思考一下，我们在查看准确性的时候，计算的到底是什么？我们目前的准确性指标只是简单地告诉我们，有多少像素计算正确。因此，在上面的例子中，我们训练 30 次，可以正确预测 98.3% 的像素值。但是，从上面的图像中可以看出，相较整个图像的大小，LV 区域通常很小。这就会产生类别失衡的问题，即一个类别的概率远高于另一个类别。在我们的例子中，如果我们只是设计一个网络，为每个输出像素输出“非 LV”类别，我们仍然会得到 95% 这样的准确性。但这会是个没用的网络。我们需要的准确性指标，要能在一定程度上表明我们的网络分割左心室图像时的表现，且不受这种失衡性的影响。

# 任务 3 -- CNN网络的Dice系数

要更准确地确定我们的神经网络在分割 LV 时的表现优劣，可以使用“Dice 系数”或“Sorensen-Dice 系数”，也可能还有其他名称。这是一个用来比较两个样本相似性的度量。在我们的例子中，会用它来比较两个目标区域，即专家标注的轮廓区域和我们预测的轮廓区域。Dice系数的计算公式是：

$$ \frac{2A_{nl}}{A_{n} + A_{l}} $$

其中 $A_n$表示我们的神经网络预测的轮廓区域，$A_l$ 表示以专业方式分割的标签所注轮廓区域，$A_{nl}$表示这二者的交叉区域，即网络预测正确的轮廓区域。1.0 代表理想分数。

由于存在类别失衡问题，这个系数可以更准确地计算出我们的神经网络在分割 LV 时的表现优劣。既然我们要确定特定轮廓中包含的区域有多大，那我们只要计算形成这个区域的像素即可。

如果您有兴趣了解如何在精度计算中加入Dice系数，可以参阅源代码文件
[`neuralnetwork.py`](../../../../edit/tasks/task1/task/exercises/cnnDice/neuralnetwork.py).

执行下面的单元 1 次，以进行训练，然后进行评估（向下两个单元），以查看准确性。然后尝试训练 30 次。这与上一个任务的操作相似。查看 30 次训练后的准确性。使用 TensorBoard 将结果可视化。

In [ ]:
!python exercises/cnnDice/runTraining.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
         --num_epochs 1

In [ ]:
!python exercises/cnnDice/runEval.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
        --eval_dir /dli/tasks/tensorboard/eval

如果训练一次，您可能会得到不足 1% 的准确性。在上一次运行中，我们得到的是

```
OUTPUT: 2017-01-27 18:44:04.103153: Dice metric = 0.034
```

（训练一次的准确性）。如果尝试训练 30 次，您可能会得到约 57% 的准确性。

```
OUTPUT: 2017-01-27 18:56:45.501209: Dice metric = 0.568
```

通过更加真实的准确性指标可以看出，这个神经网络有些地方需要改进。

# 参数搜索

现在，我们已经建立了一个神经网络，其合适的结构可以较好地完成工作，我们还使用了准确性指标来了解网络在学习分割任务时的表现。但我们的评估准确性仍不理想。接下来要考虑的问题是，我们应该试着多搜索一下参数空间。到目前为止我们修改了训练次数，但也只做了这项调整。我们还可以测试更多能够提高准确性的参数。包括：

* --learning_rate：初始学习率
* --decay_rate：初始学习率的衰减率，例如1.0 代表无衰减，0.5 代表每计算decay_steps步的学习率减半等。
* --decay_steps：修改学习率前要执行的步数。


学习率指的是每次执行反向传播时权重的调整速度。如果学习率过大，我们可能会按过大的值来调整权重，最终导致我们在正确解附近振荡，而不能收敛。如果学习率过小，权重调整量也会过小，我们可能需要很长时间后才能收敛至满意的解。这里有一个常用的方法就是将学习率设置为变量，或称“可调学习率”。刚开始训练时，我们会使用一个较大的学习率，这样可以对权重做较大调整，希望可以快速到达最优解附近，然后降低学习率继续训练，直到收敛。上述三个参数有助于您控制学习率、学习率的衰减以及每次衰减的计算步数。如果您没有选择这些选项，使用的默认值为（您到目前为止在本实验中一直使用的默认值）：

```
--learning_rate 0.01
--decay_rate 1.0
--decay_steps 1000
--num_epochs 1
```

在下一个单元中运行训练，使用这些值，看看能否得到比先前更高的准确性。由于本实验时间有限，我们不建议epochs值高于 100 以上，但在生产环境下，您很可能会运行更大的epochs值。

为方便起见，在开始训练后，如果您发现训练次数过大，可以中断运行，仍然通过运行评估（向下两个单元）来测试模型。TensorFlow 具有检查点功能，可以定期创建模型快照，这样在您中断训练后，最近的快照仍会保留下来。


In [ ]:
!python exercises/cnnDice/runTraining.py --data_dir /dli/data/img_segmentation  --checkpoint_dir /dli/tasks/tensorboard/train \
        --num_epochs 1 --learning_rate 0.01 --decay_rate 1.0 --decay_steps 1000

In [ ]:
!python exercises/cnnDice/runEval.py --data_dir /dli/data/img_segmentation --checkpoint_dir /dli/tasks/tensorboard/train \
        --eval_dir /dli/tasks/tensorboard/eval

我们使用这组参数得到的准确率为 86%。查看参考资料[A](#A)，查看我们在训练中使用的参数。

# 继续改进

为便于说明，我们着重介绍了几个可以在本实验规定时间内运行的小型任务，如果要在生产环境下运行图像分割任务，我们还需要哪些操作？我们还可执行如下操作。

* 延长训练时间 -- 我们在这里运行的训练时间很短，现实中要运行更多次数。
* 增加训练数据 -- 我们的训练集中只有 236 个图像。我们可以收集更多数据，也可以扩增已有的数据。TensorFlow 具有内置函数，可以自动翻转/旋转/转置图像。
* 更大型的网络 -- 我们可以尝试使用 AlexNet 或其他大型 CNN，并将其转化为 FCN


## 总结

在本实验中，您有机会使用 TensorFlow 框架运行图像分割的方法。您已学习如何将标准 CNN 转化为 FCN，并将其用作分割网络。还了解到选择正确的准确性指标对训练网络的重要性。最后，您还有机会了解执行参数搜索是深度学习工作流程中不可或缺的部分，这有助于最终建立起网络，以可接受的准确性执行相关任务。

## 了解更多内容

如果您有兴趣学习更多内容，可以使用下列资源：

* 在 [CUDA Developer Zone](https://developer.nvidia.com/category/zone/cuda-zone).中了解更多内容。
* 如果您的系统配有 NVIDIA GPU，可以下载并安装 [CUDA tookit](https://developer.nvidia.com/cuda-toolkit)。
* 参加精彩的线上免费课程：Udacity [Intro to Parallel Programming](https://www.udacity.com/course/cs344)课程，该课程使用 CUDA C。
* 使用 cuda 标签在[Stackoverflow](http://stackoverflow.com/questions/tagged/cuda)上搜索或提问。


<a id="post-lab"></a>
## 实验后步骤

最后，请不要忘记在时间用完和实例关闭之前保存您在本实验中的工作！

1. 要保存 IPython Notebook，可以在本窗口的顶部依次执行 `File -> Download as -> IPython (.ipynb)`
2. 您可以执行下列单元块以创建包含所使用文件的 zip 文件，并使用下面的链接下载此文件。

In [ ]:
!tar -czf ImageSegmentation.tar.gz exercise_solutions/ exercises/ images/*png

[ImageSegmentation.tar.gz](./ImageSegmentation.tar.gz)

<a id="FAQ"></a>
---
# 实验 FAQ

问：在执行单元格时碰到问题或遇到其他技术问题时，该怎么办？<br>
答：请参阅[此](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) 基础架构 FAQ。<br>
问：我在运行任务时遇到意外行为（即输出错误）。<br>
答：一次或多次 CUDA Runtime API 调用确实可能返回错误。关于 CUDA Runtime 错误，您是否遇到过屏幕上显示的任何错误？<br>

<a id="References"></a>
# 参考资料

<a id="1"></a>
[1] 早先竞赛中使用的 Sunnybrook 心脏图像 https://smial.sri.utoronto.ca/LV_Challenge/Data.html

<a id="2"></a>
[2] 这个 “Sunnybrook Cardiac MR Database”是依上述 CC0 1.0 Universal 许可提供，可于此处获取更多详情： http://creativecommons.org/publicdomain/zero/1.0/

<a id="3"></a>
[3] 出处：Radau P、Lu Y、Connelly K、Paul G、Dick AJ、Wright GA。"Evaluation Framework for Algorithms Segmenting Short Axis Cardiac MRI."（适用于短轴心脏 MRI 图像分割算法的评估框架）The MIDAS Journal -Cardiac MR Left Ventricle Segmentation Challenge（心脏 MR 左心室图像分割挑战赛）， http://hdl.handle.net/10380/3070

<a id="4"></a>
[4] http://fcn.berkeleyvision.org/

<a id="5"></a>
[5] Long, Shelhamer, Darrell; "Fully Convoutional Networks for Semantic Segmentation"（用于语义分割的全卷积网络），CVPR 2015.

<a id="6"></a>
[6] Zeiler, Krishnan, Taylor, Fergus; "Deconvolutional Networks"（反卷积网络）, CVPR 2010.

<a id="7"></a>
[7] https://www.kaggle.com/c/second-annual-data-science-bowl/details/deep-learning-tutorial


# 参考配置

<a id="A"></a>
[A] 下列配置会得到约 86% 的准确性。

```
--learning_rate 0.03
--decay_rate 0.75
--num_epochs 100
--decay_steps 10000
OUTPUT: 2017-01-27 20:19:08.702868: Dice metric = 0.862
```

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>